# Sheet 6

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.sparse
from sklearn.linear_model import Ridge, LinearRegression

## 2 Estimating Parameter Relevance

In [2]:
# load the data
with open('data/vostok.txt', 'r') as f:
    lines = f.readlines()
    
# remove header and split lines
lines = [l.split() for l in lines[2:]]

# filter out lines with missing data
lines = [l for l in lines if len(l) == 4]

# convert to float
lines = np.array(lines).astype(np.float32)
print(f'{lines.shape=}')

features = np.concatenate([lines[:, :1], lines[:, 2:]], axis=1).T
feature_names = 'age', 'CO₂', 'dust'
labels = lines[:, 1]
label_name = '∆T'

print(f'{features.shape=}, {labels.shape=}')

lines.shape=(3729, 4)
features.shape=(3, 3729), labels.shape=(3729,)


In [41]:
from sklearn.linear_model import LinearRegression as LR



# TODO: fit the linear regressor and compute the sum of square deviations\

model =  LR()   ##first creating a model
model.fit(features.T,labels)   ##this will fit the model with the input features.T and target variable labels

y_pred = model.predict(features.T) ### this is the predicted labels


def squared_residuls(pred):
    return np.sum(np.square(pred-labels))
square_error_base = squared_residuls(y_pred)
print(" The sum of squared residuals is - " +str(square_error_base))    


 The sum of squared residuals is - 6362.9375


In [43]:
# TODO: for each feature, randomly permute it amongst the samples, 

def permutation(i):
    """
    Input : takes the three values of the particular row i =[0,1,2]
    output: return the feature matrix with ith row permuted

    """
    permutation_array = np.arange(0,len(labels)) ##this will create the index array ranging from 0 to N-1 

    np.random.shuffle(permutation_array)
    X_i = np.array(features)

    X_i[i,:] = features[i,:][permutation_array]   ##permuting the ith row

    return  X_i

#       refit the regressor and compte sum of squared deviations
square_res_arr = [square_error_base]
for i in range(3):
    X = permutation(i)
    model.fit(X.T,labels)
    y_pred = model.predict(X.T)
    epsilon = squared_residuls(y_pred)
    square_res_arr.append(epsilon)
square_res_arr    







[6362.9375, 6817.8086, 19161.924, 6528.2812]

In [37]:
model2 = LR()


model2.fit(X_1.T,labels)

ypre = model2.predict(X_1.T)

squared_residuls(ypre)


6818.048

## 4 Visualize Regularization Contours

In [ ]:
# load the data
data = np.load('data/linreg.npz')
x = data['X']
y = data['Y']
print(f'{x.shape} {y.shape}')

In [ ]:
# TODO: create a grid of points in the parameter space

(a)

In [ ]:
# TODO: make coutour plots for ridge and lasso regularization terms


(b)

In [ ]:
# TODO: for each combination of parameters, compute the sum of squared deviations.
#       do not use loops, but numpy broadcasting!
# TODO: make a coutour plot for sum of squared deviations


(c)

In [ ]:
# TODO: for each lambda, plot both ridge regression and lasso loss functions
lambdas = [0, 10, 50, 100, 200, 300]


## CT

set up design matrix (run this once to save to disk)

In [ ]:
# create design matrix
# don't change any of this, just run it once to create and save the design matrix
import os

if not os.path.exists('data/design_matrix.npy'):
    res = (99, 117)
    xs = np.arange(0, res[1]+1) - res[1]/2 # np.linspace(-1, 1, res[1] + 1)
    ys = np.arange(0, res[0]+1) - res[0]/2 #np.linspace(-1, 1, res[0] + 1)

    # rays are defined by origin and direction
    n_parallel_rays = 70 
    ray_offset_range = [-res[1]/1.5, res[1]/1.5]
    n_ray_angles = 30
    n_rays = n_parallel_rays * n_ray_angles

    ray_angles = np.linspace(0, np.pi, n_ray_angles, endpoint=False) + np.pi/n_ray_angles
    
    # offsets for ray_angle = 0, i.e. parallel to x-axis
    ray_0_offsets = np.stack([np.zeros(n_parallel_rays), np.linspace(*ray_offset_range, n_parallel_rays)], axis=-1)
    ray_0_directions = np.stack([np.ones(n_parallel_rays), np.zeros(n_parallel_rays)], axis=-1)

    def rot_mat(angle):
        c, s = np.cos(angle), np.sin(angle)
        return np.stack([np.stack([c, s], axis=-1), np.stack([-s, c], axis=-1)], axis=-1)

    ray_rot_mats = rot_mat(ray_angles)

    ray_offsets = np.einsum('oi,aij->aoj', ray_0_offsets, ray_rot_mats).reshape(-1, 2)
    ray_directions = np.einsum('oi,aij->aoj', ray_0_directions, ray_rot_mats).reshape(-1, 2)

    sigma = 1
    kernel = lambda x: np.exp(-x**2/sigma**2/2)

    xsc = (xs[1:] + xs[:-1]) / 2
    ysc = (ys[1:] + ys[:-1]) / 2
    b = np.stack(np.meshgrid(xsc, ysc), axis=-1).reshape(-1, 2)
    a = ray_offsets
    v = ray_directions
    v = v / np.linalg.norm(v, axis=-1, keepdims=True)
    p = ((b[None] - a[:, None]) * v[:, None]).sum(-1, keepdims=True) * v[:, None] + a[:, None]
    d = np.linalg.norm(b - p, axis=-1)
    d = kernel(d)
    design_matrix = d.T

    np.save('data/design_matrix.npy', design_matrix)
    print(f'created and saved design matrix of shape {design_matrix.shape} at data/design_matrix.npy')

(a)

In [ ]:
design_matrix = np.load('data/design_matrix.npy')

# TODO: visualize four random columns as images, using an image shape of (99, 117)
img_shape = (99, 117)

fig, axs = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in zip(np.random.choice(np.arange(design_matrix.shape[1]), 4), axs):
    ax.imshow(design_matrix[:, i].reshape(*res));
    ax.set_title(f'column {i}')

In [ ]:
sino = np.load('data/sino.npy')

# visualize sinogram as image
n_parrallel_rays = 70
n_angles = 30
plt.imshow(sino.reshape(n_angles, n_parallel_rays), origin='lower')
plt.show();

(b)

In [ ]:
# TODO: solve the reconstruction with linear regression and visualize the result


In [ ]:
# TODO: solve the reconstruction with ridge regression and visualize the result
# Optional: try out different regularization strengths and oberve the influence
